In [0]:
!pip install tensorflow-gpu==1.15.0
import os
os.kill(os.getpid(), 9)

     |████████████████████████████████| 411.5MB 43kB/s 
     |████████████████████████████████| 512kB 37.5MB/s 
     |████████████████████████████████| 3.8MB 43.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=d9fa98d26296925fac9a3694a28f726fb14725be70ea19db5d037e4f65290d18
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninst

In [1]:
!pip install -q gpt_2_simple
import gpt_2_simple as gpt2
gpt2.download_gpt2(model_name="117M")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 531Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 98.5Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 364Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 186Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 259Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 134Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 131Mit/s]                                                       


In [0]:
import os
import json
import regex as re
from functools import lru_cache
import tensorflow as tf
import numpy as np
from tensorflow.contrib.training import HParams

## Flags and Utils


In [17]:
class FLAGS():
  def __init__(self, is_train = 1, cond = 1, model_dir = None, gpu = '0', batch_size = 10, data_name = 'kg', n_class = 4, learning_rate = 1e-4, data_dir = "data",length = 200, temperature = 0.7, top_k = 40):
    self.is_train = bool(is_train)
    self.cond = bool(cond)
    self.model_dir = model_dir
    self.gpu = gpu
    self.batch_size = batch_size
    self.data_name = data_name
    self.n_class = n_class
    self.learning_rate = 1e-4
    self.data_dir = data_dir
    self.length = length
    self.temperature = temperature
    self.top_k = top_k

FLAGS = FLAGS(model_dir = 'models/117M',batch_size = 50)
FLAGS.is_train

True

In [0]:
model_dir = os.path.expanduser(os.path.expandvars(FLAGS.model_dir))
enc = get_encoder(model_dir)
PAD_ID = enc.encoder['<|endoftext|>']
hparams = default_hparams()
with open(os.path.join(model_dir, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))

def load_data(path, fname, enc, label):
    data = []
    print('loading %s/%s ......' % (path, fname))
    with open('%s/%s.txt' % (path, fname)) as f:
        tmp = []
        for k, line in enumerate(f):
            i = k + 1
            if i % 6 == 0:
                data.append({"st": tmp, "label": label})
                tmp = []
            else:
                tmp.append(enc.encode(line.strip().replace(" .", ". ")))
    return data

def load_data_kg(path, fname, enc):
    with open("%s/%s.txt" % (path, fname)) as fin:
        data = [enc.encode(line.strip()) for line in fin]
    return data

def padding(sent, l):
    return sent + [PAD_ID] * (l-len(sent))

# def gen_batched_data(data):
#     max_story_len = max([sum([len(item["st"][i]) for i in range(5)]) for item in data]) + 1
#     max_input_story_len = max([len(item["st"][0]) for item in data]) + 1
#     story, story_length, label = [], [], []
#     input_story, input_story_length = [], []

#     for item in data:
#         input_story.append(padding(item["st"][0], max_input_story_len))
#         input_story_length.append(len(item["st"][0]) + 1)    
#         story.append([])
#         for i in range(5):
#             story[-1] += item["st"][i]
#         story_length.append(len(story[-1]) + 1)
#         story[-1] = padding(story[-1], max_story_len)
#         label.append(item["label"])
#     batched_data = {
#         "story": np.array(story),
#         "story_length": np.array(story_length),
#         "input_story": np.array(input_story),
#         "input_story_length": np.array(input_story_length),
#         "label": np.array(label),
#     }
#     return batched_data


def gen_batched_data_from_kg(data):
    max_story_len = max([len(item) for item in data]) + 1
    story, story_length = [], []
    for item in data:
        story.append(padding(item, max_story_len))
        story_length.append(len(item) + 1)
    batched_data = {
        "story": np.array(story),
        "story_length": np.array(story_length),
    }
    return batched_data

## Models


In [0]:
def default_hparams():
    return HParams(
        n_vocab=0,
        n_ctx=1024,
        n_embd=768,
        n_head=12,
        n_layer=12,
    )

def shape_list(x):
    """Deal with dynamic shape in tensorflow cleanly."""
    static = x.shape.as_list()
    dynamic = tf.shape(x)
    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

def softmax(x, axis=-1):
    x = x - tf.reduce_max(x, axis=axis, keepdims=True)
    ex = tf.exp(x)
    return ex / tf.reduce_sum(ex, axis=axis, keepdims=True)

def gelu(x):
    return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))

def norm(x, scope, *, axis=-1, epsilon=1e-5):
    """Normalize to mean = 0, std = 1, then do a diagonal affine transform."""
    with tf.variable_scope(scope):
        n_state = x.shape[-1].value
        g = tf.get_variable('g', [n_state], initializer=tf.constant_initializer(1))
        b = tf.get_variable('b', [n_state], initializer=tf.constant_initializer(0))
        u = tf.reduce_mean(x, axis=axis, keepdims=True)
        s = tf.reduce_mean(tf.square(x-u), axis=axis, keepdims=True)
        x = (x - u) * tf.rsqrt(s + epsilon)
        x = x*g + b
        return x

def split_states(x, n):
    """Reshape the last dimension of x into [n, x.shape[-1]/n]."""
    *start, m = shape_list(x)
    return tf.reshape(x, start + [n, m//n])

def merge_states(x):
    """Smash the last two dimensions of x into a single dimension."""
    *start, a, b = shape_list(x)
    return tf.reshape(x, start + [a*b])

def conv1d(x, scope, nf, *, w_init_stdev=0.02):
    with tf.variable_scope(scope):
        *start, nx = shape_list(x)
        w = tf.get_variable('w', [1, nx, nf], initializer=tf.random_normal_initializer(stddev=w_init_stdev))
        b = tf.get_variable('b', [nf], initializer=tf.constant_initializer(0))
        c = tf.reshape(tf.matmul(tf.reshape(x, [-1, nx]), tf.reshape(w, [-1, nf]))+b, start+[nf])
        return c

def attention_mask(nd, ns, *, dtype):
    """1's in the lower triangle, counting from the lower right corner.

    Same as tf.matrix_band_part(tf.ones([nd, ns]), -1, ns-nd), but doesn't produce garbage on TPUs.
    """
    i = tf.range(nd)[:,None]
    j = tf.range(ns)
    m = i >= (j - ns + nd)
    return tf.cast(m, dtype)

def attention_mask_s2s(nd, ns, *, dtype):
    """1's in the lower triangle, counting from the lower right corner.

    Same as tf.matrix_band_part(tf.ones([nd, ns]), -1, ns-nd), but doesn't produce garbage on TPUs.
    """
    i = tf.range(nd)[:,None]
    j = tf.range(ns)
    m = i >= (j - ns + nd)
    return tf.cast(m, dtype)


def attn(x, scope, n_state, *, past, hparams):
    assert x.shape.ndims == 3  # Should be [batch, sequence, features]
    assert n_state % hparams.n_head == 0
    if past is not None:
        assert past.shape.ndims == 5  # Should be [batch, 2, heads, sequence, features], where 2 is [k, v]

    def split_heads(x):
        # From [batch, sequence, features] to [batch, heads, sequence, features]
        return tf.transpose(split_states(x, hparams.n_head), [0, 2, 1, 3])

    def merge_heads(x):
        # Reverse of split_heads
        return merge_states(tf.transpose(x, [0, 2, 1, 3]))

    def mask_attn_weights(w):
        # w has shape [batch, heads, dst_sequence, src_sequence], where information flows from src to dst.
        batch_size, _, nd, ns = shape_list(w)
        b = attention_mask(nd, ns, dtype=w.dtype,)
        b = tf.reshape(b, [1, 1, nd, ns])
        w = w*b - tf.cast(1e10, w.dtype)*(1-b)
        return w

    def multihead_attn(q, k, v):
        # q, k, v have shape [batch, heads, sequence, features]
        w = tf.matmul(q, k, transpose_b=True)
        # w = tf.Print(w, [tf.shape(q), tf.shape(k), tf.shape(v), tf.shape(w)], summarize=1e5)
        w = w * tf.rsqrt(tf.cast(v.shape[-1].value, w.dtype))

        w = mask_attn_weights(w)
        w = softmax(w)
        a = tf.matmul(w, v)
        return a

    with tf.variable_scope(scope):
        c = conv1d(x, 'c_attn', n_state*3)
        q, k, v = map(split_heads, tf.split(c, 3, axis=2))
        present = tf.stack([k, v], axis=1)
        if past is not None:
            pk, pv = tf.unstack(past, axis=1)
            k = tf.concat([pk, k], axis=-2)
            v = tf.concat([pv, v], axis=-2)
        a = multihead_attn(q, k, v)
        a = merge_heads(a)
        a = conv1d(a, 'c_proj', n_state)
        return a, present


def mlp(x, scope, n_state, *, hparams):
    with tf.variable_scope(scope):
        nx = x.shape[-1].value
        h = gelu(conv1d(x, 'c_fc', n_state))
        h2 = conv1d(h, 'c_proj', nx)
        return h2


def block(x, scope, *, past, hparams):
    with tf.variable_scope(scope):
        nx = x.shape[-1].value
        a, present = attn(norm(x, 'ln_1'), 'attn', nx, past=past, hparams=hparams)
        x = x + a
        m = mlp(norm(x, 'ln_2'), 'mlp', nx*4, hparams=hparams)
        x = x + m
        return x, present

def past_shape(*, hparams, batch_size=None, sequence=None):
    return [batch_size, hparams.n_layer, 2, hparams.n_head, sequence, hparams.n_embd // hparams.n_head]

def expand_tile(value, size):
    """Add a new axis of given size."""
    value = tf.convert_to_tensor(value, name='value')
    ndims = value.shape.ndims
    return tf.tile(tf.expand_dims(value, axis=0), [size] + [1]*ndims)

def positions_for(tokens, past_length):
    batch_size = tf.shape(tokens)[0]
    nsteps = tf.shape(tokens)[1]
    return expand_tile(past_length + tf.range(nsteps), batch_size)

def transformer(h, hparams, past):
    presents = []
    pasts = tf.unstack(past, axis=1) if past is not None else [None] * hparams.n_layer
    assert len(pasts) == hparams.n_layer
    for layer, past in enumerate(pasts):
        h, present = block(h, 'h%d' % (layer), past=past, hparams=hparams)
        presents.append(present)
    return h, presents

def model(hparams, X, past=None, kg=None, kg_mask=None, kg_past=None, scope='model', reuse=False):
    with tf.variable_scope(scope, reuse=reuse):
        results = {}
        batch, sequence = shape_list(X)
        wpe = tf.get_variable('wpe', [hparams.n_ctx, hparams.n_embd],
                             initializer=tf.random_normal_initializer(stddev=0.01))
        wte = tf.get_variable('wte', [hparams.n_vocab, hparams.n_embd],
                             initializer=tf.random_normal_initializer(stddev=0.02))
        past_length = 0 if past is None else tf.shape(past)[-2]
        h = tf.gather(wte, X) + tf.gather(wpe, positions_for(X, past_length))
        print_op = tf.print(["x:", batch, sequence, tf.shape(h)])
        with tf.control_dependencies([]):
            # Transformer
            h, presents = transformer(h, hparams, past)
        results['present'] = tf.stack(presents, axis=1)
        h = norm(h, 'ln_f')

    # Language model loss.  Do tokens <n predict token n?
    h_flat = tf.reshape(h, [batch*sequence, hparams.n_embd])
    results['hidden_state'] = tf.reshape(h_flat, [batch, sequence, hparams.n_embd])
    logits = tf.matmul(h_flat, wte, transpose_b=True)
    logits = tf.reshape(logits, [batch, sequence, hparams.n_vocab])
    results['logits'] = logits
    return results

def model_train(*, hparams, context, context_length, enc, PAD_ID):
    batch_size = tf.shape(context)[0]
    decoder_length = tf.shape(context)[1]

    context_input = tf.concat([tf.ones([batch_size, 1], dtype=tf.int32)*PAD_ID,
        tf.split(context, [decoder_length-1, 1], 1)[0]], 1)   # batch*len
    context_mask = tf.reshape(tf.cumsum(tf.one_hot(context_length - 1,
        decoder_length), reverse=True, axis=1), [-1, decoder_length])

    lm_output = model(hparams=hparams, X=context_input, past=None, reuse=tf.AUTO_REUSE)
    logits = lm_output["logits"]
    labels_onehot = tf.clip_by_value(tf.one_hot(context, hparams.n_vocab), 0.0, 1.0)
    local_loss = tf.reduce_sum(-labels_onehot * tf.nn.log_softmax(logits), 2) * context_mask
    loss = tf.reduce_sum(local_loss)
    total_size = tf.reduce_sum(context_mask)
    total_size += 1e-12 # to avoid division by 0 for all-0 weights
    return loss / total_size

def train_classify(*, hparams, context, context_length, label, enc, n_class, PAD_ID):
    batch_size = tf.shape(context)[0]
    decoder_length = tf.shape(context)[1]

    context_input = tf.concat([tf.ones([batch_size, 1], dtype=tf.int32)*PAD_ID,
        tf.split(context, [decoder_length-1, 1], 1)[0]], 1)   # batch*len
    context_mask = tf.reshape(tf.cumsum(tf.one_hot(context_length-1,
        decoder_length), reverse=True, axis=1), [-1, decoder_length])

    lm_output = model(hparams=hparams, X=context_input, past=None, reuse=tf.AUTO_REUSE)

    logits = tf.reduce_sum(lm_output["hidden_state"]*tf.expand_dims(context_mask, 2), 1) / (tf.reduce_sum(tf.expand_dims(context_mask, 2), 1) + 1e-12)
    logits = tf.layers.dense(logits, 4, name="fine_tuning/classify")
    local_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(label, 4), logits=logits)
    loss = tf.reduce_sum(local_loss)

    acc_list = []
    for l in range(n_class):
        tmp_acc_mask = tf.cast(tf.equal(label, l), tf.float32)
        for tmpl in range(n_class):
            tmp_is_equal = tf.cast(tf.equal(tf.cast(tf.argmax(logits, 1), tf.int32), tmpl), tf.float32)
            acc_list.append(tf.reduce_sum(tmp_is_equal*tmp_acc_mask) / (tf.reduce_sum(tmp_acc_mask)+1e-12))

    labels_onehot = tf.clip_by_value(tf.one_hot(context, hparams.n_vocab), 0.0, 1.0)
    # [batch_size, 1]
    label_mask = tf.expand_dims(tf.cast(tf.equal(label, 0), tf.float32), 1)
    logits = lm_output["logits"]
    local_loss_lm = tf.reduce_sum(-labels_onehot * tf.nn.log_softmax(logits), 2) * context_mask * label_mask
    loss_lm = tf.reduce_sum(local_loss_lm)
    total_size = tf.reduce_sum(context_mask * label_mask)
    total_size += 1e-12 # to avoid division by 0 for all-0 weights
    loss_lm /= total_size
    return 0.05*loss+loss_lm, loss_lm, acc_list


##Encoder

In [0]:
"""Byte pair encoding utilities"""

@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a signficant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))

def get_pairs(word):
    """Return set of symbol pairs in a word.

    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

class Encoder:
    def __init__(self, encoder, bpe_merges, errors='replace'):
        self.encoder = encoder
        self.decoder = {v:k for k,v in self.encoder.items()}
        self.errors = errors # how to handle errors in decoding
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v:k for k, v in self.byte_encoder.items()}
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
        self.cache = {}

        # Should haved added re.IGNORECASE so BPE merges can happen for capitalized versions of contractions
        self.pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token)
        pairs = get_pairs(word)

        if not pairs:
            return token

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        bpe_tokens = []
        for token in re.findall(self.pat, text):
            # print("before processing:", token)
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            # print("after processing:", token, self.bpe(token), [self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' ')])
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))
        return bpe_tokens

    def decode(self, tokens, trunct=False):
        text = ''.join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode('utf-8', errors=self.errors)
        if trunct:
            text = text[:text.find('<|endoftext|>')]
        return text

def get_encoder(model_dir):
    with open(os.path.join(model_dir, 'encoder.json'), 'r') as f:
        encoder = json.load(f)
    with open(os.path.join(model_dir, 'vocab.bpe'), 'r', encoding="utf-8") as f:
        bpe_data = f.read()
    bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
    return Encoder(
        encoder=encoder,
        bpe_merges=bpe_merges,
    )


## generate_unconditional_sample

In [0]:
def sample_model(
    sess,
    enc,
    PAD_ID,
    hparams,
    temperature=1,
    top_k=0,
):
    """
    Run the sample_model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to
     reproduce results
    :nsamples=0 : Number of samples to return, if 0, continues to
     generate samples indefinately.
    :batch_size=1 : Number of batches (only affects speed/memory).
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)
    """

    length = hparams.n_ctx // 2 if FLAGS.length is None else FLAGS.length
    if length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    output = sample.sample_sequence(
        hparams=hparams, length=length,
        start_token=PAD_ID,
        batch_size=1,
        temperature=temperature, top_k=top_k
    )[:, 1:]

    generated = 0
    while generated < 1000:
        out = sess.run(output)
        generated += 1
        text = enc.decode(out[0], trunct=True)
        print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
        print(text)


## Interactive generate samples

In [0]:
def interact_model(
    sess,
    enc,
    PAD_ID,
    hparams,
    context,
    dataset=None,
    output_file_name=None,
    relation=None,
    temperature=1,
    top_k=0,
):
    """
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)     
    """

    if FLAGS.length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)
    batch_size = 1
    output = sample.sample_sequence(
        hparams=hparams, length=FLAGS.length,
        context=context,
        start_token=PAD_ID,
        batch_size=1,
        temperature=temperature,
        top_k=top_k,
    )
    if dataset is None:
        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            out = sess.run(output, feed_dict={
                context: [context_tokens for _ in range(batch_size)]
            })[:, len(context_tokens):]
            text = enc.decode(out[0], trunct=True).strip()
            print(text)
            print("=" * 80)
    else:
        fout = open(output_file_name, "w")
        st, ed = 0, 0
        while ed < len(dataset):
            st, ed = ed, ed + 1
            context_tokens = [[PAD_ID] + dataset[st]["st"][0]]
            out = sess.run(output, feed_dict={
                context: context_tokens,
            })
            for ipt, opt in zip(context_tokens, out):
                opt = enc.decode(opt[len(ipt):], trunct=True)
                ipt = enc.decode(ipt[1:], trunct=False)
                fout.write("ipt: " + ipt + "\n")
                fout.write("opt: " + opt + "\n")
                fout.write("-"*5+"\n")
        fout.close()

## sample

In [0]:
def top_k_logits(logits, k):
    if k == 0:
        # no truncation
        return logits

    def _top_k():
        values, _ = tf.nn.top_k(logits, k=k)
        min_values = values[:, -1, tf.newaxis]
        return tf.where(
            logits < min_values,
            tf.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )
    return tf.cond(
       tf.equal(k, 0),
       lambda: logits,
       lambda: _top_k(),
    )


def sample_sequence(*, hparams, length, start_token=None, batch_size=None, context=None, temperature=1, top_k=0):
    if context is not None:
        context_input = context
        encoder_length = tf.shape(context)[1]
    else:
        context_input = tf.fill([batch_size, 1], start_token)

    def step(hparams, tokens, past=None):
        lm_output = model.model(hparams=hparams, X=tokens, past=past, reuse=tf.AUTO_REUSE)

        logits = lm_output['logits'][:, :, :hparams.n_vocab]
        presents = lm_output['present']
        presents.set_shape(model.past_shape(hparams=hparams, batch_size=batch_size))
        return {
            'logits': logits,
            'presents': presents,
        }

    with tf.name_scope('sample_sequence'):
        def body(past, prev, output):
            next_outputs = step(hparams, prev, past=past)
            logits = next_outputs['logits'][:, -1, :]  / tf.to_float(temperature)
            logits = top_k_logits(logits, k=top_k)
            samples = tf.multinomial(logits, num_samples=1, output_dtype=tf.int32)
            return [
                next_outputs['presents'] if past is None else tf.concat([past, next_outputs['presents']], axis=-2),
                samples,
                tf.concat([output, samples], axis=1)
            ]
        # past: the past hidden states of the transformer
        # prev: the prediction token
        # output: the cumulative tokens
        past, prev, output = body(None, context_input, context_input)
        prev = tf.reshape(prev, [batch_size, 1])
        output = tf.reshape(output, [batch_size, -1])

        def cond(*args):
            return True

        _, _, tokens = tf.while_loop(
            cond=cond, body=body,
            maximum_iterations=length - 1,
            loop_vars=[
                past,
                prev,
                output
            ],
            shape_invariants=[
                tf.TensorShape(model.past_shape(hparams=hparams, batch_size=batch_size)),
                tf.TensorShape([batch_size, None]),
                tf.TensorShape([batch_size, None]),
            ],
            back_prop=False,
        )

        return tokens


## Main

In [18]:
import time
import copy
import random
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = FLAGS.gpu
print("Using %s-th gpu ..." % os.environ["CUDA_VISIBLE_DEVICES"])
train_dir = os.path.join(FLAGS.model_dir)
assert os.path.exists(train_dir)

def train(sess, dataset, is_train=True):
    def pro_acc(acc):
        final_acc = [[] for _ in range(FLAGS.n_class)]
        for ac in acc:
            for i in range(4):
                if np.sum(ac[i*4:(i+1)*4]) == 1:
                    final_acc[i].append(ac[i*4:(i+1)*4])
        for i in range(4):
            print("final classification confusion matrix (%d-th category):" % i, np.mean(final_acc[i], 0).tolist())
    st, ed = 0, 0
    loss, loss_lm, acc = [], [], []
    while ed < len(dataset):
        if is_train:
            output_feed = [
                model_loss,
                gradient_norm,
                update,
            ]
        else:
            output_feed = [
                model_loss,
            ]
        st, ed = ed, ed + FLAGS.batch_size if ed + \
            FLAGS.batch_size < len(dataset) else len(dataset)
        if FLAGS.data_name == "multi_roc" or FLAGS.data_name == "roc":
            batch_data = gen_batched_data(dataset[st:ed])
            input_feed = {
                context: batch_data["story"],
                context_length: batch_data["story_length"],
                label: batch_data["label"],
            }
            if FLAGS.data_name == "multi_roc": 
                output_feed.append(model_loss_lm)
                output_feed.append(model_acc_list)
        elif FLAGS.data_name == "kg":
            batch_data = gen_batched_data_from_kg(dataset[st:ed])
            input_feed = {
                context: batch_data["story"],
                context_length: batch_data["story_length"],
            }
        else:
            print("DATANAME ERROR")
        outputs = sess.run(output_feed, input_feed)
        loss.append(outputs[0])
        if FLAGS.data_name == "multi_roc":
            loss_lm.append(outputs[-2])
            acc.append(outputs[-1])
            if (st+1) % 10000 == 0:
                print("current ppl:%.4f"%np.exp(np.mean(loss_lm)))
                pro_acc(acc)
                print("="*5)
    if is_train:
        sess.run(epoch_add_op)
    if FLAGS.data_name == "multi_roc":
        pro_acc(acc)
        return np.exp(np.mean(loss_lm))
    else:
        return np.exp(np.mean(loss))

#**********************************************************************************
#**********************************************************************************
#**********************************************************************************

print("begin loading dataset......")
if FLAGS.data_name == "roc":
    data = load_data(FLAGS.data_dir, FLAGS.data_name, enc, label=0)
    data_segnum = len(data) / 20
    data_train = data[:int(data_segnum*18)]
    data_dev = data[int(data_segnum*18):int(data_segnum*19)]
    data_test = data[int(data_segnum*19):]
elif FLAGS.data_name == "kg":
    data_train = load_data_kg(FLAGS.data_dir, 'train', enc)
    data_dev = load_data_kg(FLAGS.data_dir, 'valid', enc)
    data_test = load_data_kg(FLAGS.data_dir, 'test', enc)
elif FLAGS.data_name == "multi_roc":
    data_train, data_dev, data_test = [], [], []
    data_name = ["roc", "roc_shuffle", "roc_replace", "roc_repeat"]
    assert FLAGS.n_class == len(data_name)
    for _, name in enumerate(data_name):
        if "shuffle" in name:
            label = 1
        elif "replace" in name:
            label = 2
        elif "repeat" in name:
            label = 3
        else:
            label = 0
        data = load_data(FLAGS.data_dir, name, enc, label=label)
        data_segnum = len(data) / 20
        data_train.extend(data[:int(data_segnum*18)])
        data_dev.extend(data[int(data_segnum*18):int(data_segnum*19)])
        data_test.extend(data[int(data_segnum*19):])
else:
    print("DATANAME ERROR")
    exit()
random.shuffle(data_train)
random.shuffle(data_dev)
random.shuffle(data_test)
print("Number of data for training:%d"%len(data_train))
print("Number of data for validation:%d"%len(data_dev))
print("Number of data for testing:%d"%len(data_test))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    context = tf.placeholder(tf.int32, [None, None])
    context_length = tf.placeholder(tf.int32, [None])
    label = tf.placeholder(tf.int32, [None])
    
    context_entity = tf.placeholder(tf.int32, [None, None, None, 3, None])
    context_entity_mask = tf.placeholder(tf.int32, [None, None, None, 3, None])

    epoch = tf.Variable(0, trainable=False, name='initialize/epoch')
    epoch_add_op = epoch.assign(epoch + 1)
    if FLAGS.data_name == "multi_roc":
        model_loss, model_loss_lm, model_acc_list = train_classify(
            hparams=hparams,
            context=context,
            context_length=context_length,
            label=label,
            enc=enc,
            n_class=FLAGS.n_class,
            PAD_ID=PAD_ID,
        )
    else:
        model_loss = model_train(
            hparams=hparams,
            context=context,
            context_length=context_length,
            enc=enc,
            PAD_ID=PAD_ID,
        )

    params = tf.trainable_variables()
    for item in params:
        print('%s: %s' % (item.name, item.get_shape()))

    # initialize the training process
    learning_rate = tf.Variable(float(FLAGS.learning_rate), trainable=False, 
            dtype=tf.float32, name="initialize/learning_rate")
    learning_rate_decay_op = learning_rate.assign(learning_rate * 0.95)
    learning_rate_assign_op = learning_rate.assign(FLAGS.learning_rate)
    global_step = tf.Variable(1, trainable=False, name="initialize/global_step")

    max_gradient_norm = 5
    clipped_gradients, gradient_norm = tf.clip_by_global_norm(tf.gradients(model_loss, params), 
            max_gradient_norm)
    update = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(clipped_gradients, params), global_step=global_step)

    try:
        saver = tf.train.Saver(tf.global_variables(), write_version=tf.train.SaverDef.V2, 
                max_to_keep=10, pad_step_number=True, keep_checkpoint_every_n_hours=1.0)
        print("Reading model parameters from %s" % (train_dir))
        saver.restore(sess, tf.train.latest_checkpoint(train_dir))
    except:
        sess.run(tf.global_variables_initializer())
        try:
            restore_tensor = []
            for tensor in tf.global_variables():
                if "fine_tuning" not in tensor.name:
                    restore_tensor.append(tensor)
                else:
                    print("to-be-initialized parameter:", tensor.name)
            print("="*5)
            saver = tf.train.Saver(restore_tensor, write_version=tf.train.SaverDef.V2, 
                    max_to_keep=10, pad_step_number=True, keep_checkpoint_every_n_hours=1.0)
            saver.restore(sess, tf.train.latest_checkpoint(train_dir))
            print("Initialize the classifier parameter.")
        except:
            restore_tensor = []
            for tensor in tf.global_variables():
                if "beta" not in tensor.name and "initialize" not in tensor.name and "fine_tuning" not in tensor.name and "Adam" not in tensor.name:
                    restore_tensor.append(tensor)
                else:
                    print("to-be-initialized parameter:", tensor.name)
            print("="*5)
            saver = tf.train.Saver(restore_tensor, write_version=tf.train.SaverDef.V2, 
                    max_to_keep=10, pad_step_number=True, keep_checkpoint_every_n_hours=1.0)
            saver.restore(sess, tf.train.latest_checkpoint(train_dir))
            print("Initialize all the fine-tuning parameter.")
        saver = tf.train.Saver(tf.global_variables(), write_version=tf.train.SaverDef.V2, 
                max_to_keep=10, pad_step_number=True, keep_checkpoint_every_n_hours=1.0)
        saver.save(sess, '%s/checkpoint' % train_dir, global_step=global_step.eval())
        print("Reading model parameters from %s and initialize the parameters for fine-tuning." % (train_dir))

    if FLAGS.is_train:
        best_loss = 1e10
        pre_losses = [1e18] * 3
        while True:
            random.shuffle(data_train)
            start_time = time.time()
            loss = train(sess, data_train, is_train=True)
            if loss > max(pre_losses):  # Learning rate decay
                sess.run(learning_rate_decay_op)
            pre_losses = pre_losses[1:] + [loss]
            print("Gen epoch %d learning rate %.4f epoch-time %.4f: " % (epoch.eval(), learning_rate.eval(), time.time() - start_time))
            print("PPL on training set:", loss)
            loss = train(sess, data_test, is_train=False)
            print("        PPL on validation set:", loss)
            if loss < best_loss:
                best_loss = loss
                loss = train(sess, data_test, is_train=False)
                print("        PPL on testing set:", loss)
                saver.save(sess, '%s/checkpoint' % train_dir, global_step=global_step.eval())
                print("saving parameters in %s" % train_dir)
    else:
        if FLAGS.cond:
            print("begin conditionally generating stories......")
            interact_model(sess=sess,
                            enc=enc,
                            PAD_ID=PAD_ID,
                            hparams=hparams,
                            context=context,
                            dataset=data_test, #  Accept console input if `dataset` is set to None 
                            output_file_name="./inference_gpt2.txt",
                            temperature=FLAGS.temperature,
                            top_k=FLAGS.top_k)
        else:
            print("begin unconditionally generating stories......")
            sample_model(sess=sess,
                            enc=enc,
                            PAD_ID=PAD_ID,
                            hparams=hparams,
                            temperature=FLAGS.temperature,
                            top_k=FLAGS.top_k)
        print("end generating stories......")

Using 0-th gpu ...
begin loading dataset......
Number of data for training:100001
Number of data for validation:1960
Number of data for testing:1987
model/wpe:0: (1024, 768)
model/wte:0: (50257, 768)
model/h0/ln_1/g:0: (768,)
model/h0/ln_1/b:0: (768,)
model/h0/attn/c_attn/w:0: (1, 768, 2304)
model/h0/attn/c_attn/b:0: (2304,)
model/h0/attn/c_proj/w:0: (1, 768, 768)
model/h0/attn/c_proj/b:0: (768,)
model/h0/ln_2/g:0: (768,)
model/h0/ln_2/b:0: (768,)
model/h0/mlp/c_fc/w:0: (1, 768, 3072)
model/h0/mlp/c_fc/b:0: (3072,)
model/h0/mlp/c_proj/w:0: (1, 3072, 768)
model/h0/mlp/c_proj/b:0: (768,)
model/h1/ln_1/g:0: (768,)
model/h1/ln_1/b:0: (768,)
model/h1/attn/c_attn/w:0: (1, 768, 2304)
model/h1/attn/c_attn/b:0: (2304,)
model/h1/attn/c_proj/w:0: (1, 768, 768)
model/h1/attn/c_proj/b:0: (768,)
model/h1/ln_2/g:0: (768,)
model/h1/ln_2/b:0: (768,)
model/h1/mlp/c_fc/w:0: (1, 768, 3072)
model/h1/mlp/c_fc/b:0: (3072,)
model/h1/mlp/c_proj/w:0: (1, 3072, 768)
model/h1/mlp/c_proj/b:0: (768,)
model/h2/ln_1/

KeyboardInterrupt: ignored